In [1]:
import pandas as pd 
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns 
import plotly.express as px
from sklearn.model_selection import train_test_split, GridSearchCV,  cross_val_score
from sklearn import preprocessing, linear_model
from sklearn.preprocessing import  LabelEncoder
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler 
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
from sklearn.linear_model import Ridge, Lasso, LinearRegression, LogisticRegression
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.metrics import make_scorer, r2_score
from lightgbm import LGBMRegressor

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [ ]:
# Read in the data

df = pd.read_csv('data.csv', sep=';')

# Data cleaning

### Diagnosis

In [ ]:
# Duplicate entries

print("Duplicate entry in data:",len(df[df.duplicated()])) 
df.drop_duplicates(inplace=True)

In [ ]:
def datainfo(df):
    temp_ps = pd.DataFrame(index=df.columns)
    temp_ps['DataType'] = df.dtypes
    temp_ps["Count"] = df.count()
    temp_ps['Unique values'] = df.nunique()
    temp_ps['Missing values'] = df.isnull().sum()
    temp_ps['Missing values percentage'] = (temp_ps['Missing values']/len(df))*100 
    df_desc = df.describe().transpose()
    temp_ps['Count'] = df_desc['count']
    temp_ps['Mean'] = df_desc['mean']
    temp_ps['Std'] = df_desc['std']
    temp_ps['Min'] = df_desc['min']
    temp_ps['25%'] = df_desc['25%']
    temp_ps['50%'] = df_desc['50%']
    temp_ps['75%'] = df_desc['75%']
    temp_ps['Max'] = df_desc['max']
    numerical_columns = df.select_dtypes(include=[np.number]).columns
    temp_ps['Skewness'] = df[numerical_columns].skew()
    temp_ps['Kurtosis'] = df[numerical_columns].kurtosis()
    return temp_ps

display(datainfo(df))

- We can see 4 categorical variables
- We can see 10 discrete variables

- No missing values 
- Low unique counts in last 3 columns

- Very heavy skewness on Rainfall and Snowfall will have to be treated

### Filtering

In [ ]:
display(df.groupby('Functioning Day').sum()['Rented Bike Count'].sort_values(ascending = False).reset_index())

# We only have data for Functioning Day = Yes, so we can drop this column and the rows where Functioning Day = No
before = len(df)
df_filtered=df.drop(df[df['Functioning Day'] == 'No'].index) 
df_filtered.drop(['Functioning Day'], axis=1, inplace = True)

# Number of rows remaining
print("Number of rows remaining:", len(df_filtered), f"({round(len(df_filtered)/before*100,2)}% of the original dataset)")

#TODO: A justifier par une visualisation
colors = ["green", "purple"]

plt.figure(figsize=(15, 8))
sns.catplot(x='Hour', y='Rented Bike Count', hue='Functioning Day', data=df, kind='point', height=5, aspect=2, palette=colors)

plt.xlabel('Hour')
plt.ylabel('Rented Bike Count')
plt.title('Hourly Distribution of Rented Bike Count based on Functioning Day')

legend_labels = ['Non-Working Day', 'Working Day']
legend_handles = [plt.Line2D([0], [0], marker='o', color=colors[i], label=label, markersize=5) for i, label in enumerate(legend_labels)]
plt.legend(handles=legend_handles, title='Functioning Day')

plt.show()

# We can see that all non-working days are always at 0, so we can drop this column and the rows where Functioning Day = No

In [ ]:
# Separate the date column into day, month and year columns

df_filtered['Date'] = pd.to_datetime(df_filtered['Date'], format='%d/%m/%Y')
df_filtered['Day'] = df_filtered['Date'].dt.day
df_filtered['Month'] = df_filtered['Date'].dt.month
df_filtered['Year'] = df_filtered['Date'].dt.year

# Create Weekend column
df['Date'] = pd.to_datetime(df['Date'], format='%d/%m/%Y')
df['Weekday'] = df['Date'].dt.day_name()
df['Weekday'] = pd.Categorical(df['Weekday'], categories=['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday'], ordered=True)
df['Weekend'] = df['Weekday'].map(lambda x: 1 if x in ['Saturday', 'Sunday'] else 0)
df.drop('Weekday', axis=1, inplace=True)

# Drop the date column

df_filtered.drop(['Date'], axis=1, inplace = True)

In [ ]:
# Resulting dataframe

df_filtered.head()

# Exploratory data analysis

Here we can see the graphs between all the columns of the dataframe, this allows us to have a first view of the correlations of the dataframe

In [ ]:
# Plotting the correlation matrix

numerical_columns = df_filtered.select_dtypes(include=[np.number]).columns
corr_matrix = df_filtered[numerical_columns].corr()
plt.figure(figsize=(12,10))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm')
plt.show()

df.columns

In [ ]:
# Dew point temperature and temperature are very highly correlated (0.91), so we can drop one of them

df_filtered.drop(['Dew point temperature(°C)'], axis=1, inplace=True)

plt.scatter(df['Temperature(°C)'], df['Dew point temperature(°C)'])
plt.xlabel('Temperature(°C)')
plt.ylabel('Dew point temperature(°C)')
plt.title('Scatter Plot of Temperature vs. Dew Point Temperature')
plt.show()

# Feature engineering

### Encoding

In [ ]:
df_encoded = df.apply(LabelEncoder().fit_transform)
df_encoded.head()

In [ ]:
plt.figure(figsize=(10,5))
plt.xticks(rotation=90) 
sns.boxplot(data=df_encoded, orient='h') 
plt.xscale('log')
plt.show()

- We can see through this the variance of features.
- There a few outliers, and they are mainly in the features Rainfall and Snowfall, so treating them would lead to removing those column's value, which isnt good.

### Visualization of Rented Bike Count VS others features

In [ ]:
plt.figure(figsize=(10, 6))
sns.barplot(x='Weekend', y='Rented Bike Count', data=df_encoded, ci=None)
plt.title('Average Rented Bikes on Weekdays vs Weekends')
plt.xlabel('Day Type')
plt.ylabel('Average Rented Bike Count')
plt.xticks([0, 1], ['Weekday', 'Weekend'])
plt.show()

We can see that the difference in bikes rented between weekends and weekdays is very small so we can remove weekends from the data ("df_encoded")

In [ ]:
# Drop the weekend column

df_encoded.drop(['Weekend'], axis=1, inplace = True)

In [ ]:
colors = plt.cm.viridis(df['Rented Bike Count'] / max(df['Rented Bike Count']))

plt.figure(figsize=(10, 5))
bars = plt.bar(df['Hour'], df['Rented Bike Count'], color=colors)

plt.xlabel('Hour')
plt.ylabel('Rented Bike Count')
plt.title('Rented Bike Count Per Hour')
plt.xticks(range(0, 24))

sm = plt.cm.ScalarMappable(cmap='viridis', norm=plt.Normalize(vmin=df['Rented Bike Count'].min(), vmax=df['Rented Bike Count'].max()))
cbar = plt.colorbar(sm, ax=plt.gca())
cbar.set_label('Intensity')

plt.show()

In [ ]:
seasons_mapping = {1: 'Spring', 2: 'Summer', 3: 'Autumn', 4: 'Winter'}

plt.figure(figsize=(15, 8))
graph = sns.lineplot(x='Hour', y='Rented Bike Count', hue='Seasons', data=df, marker='x', markeredgecolor='black')
plt.xticks(range(0, 24))
plt.xlabel('Hour')
plt.ylabel('Rented Bike Count')
plt.title('Hourly Distribution of Rented Bike Count for Each Season')

plt.show()

The visualization shows that the values are arranged in the following order (in ascending order): Winter, Spring, Autumn, and Summer. This indicates that the demand for rented bikes tends to be lower during the winter season, followed by a gradual increase in the spring and autumn seasons. The highest values are observed during the summer season, possibly due to favorable weather conditions and increased outdoor activities. It seems that weather conditions affect the number of bike rentals

The highest peak in bike rentals occurs between 4 PM and 8 PM (6 PM is the highest), indicating a significant demand during the evening hours. A notable spike is also observed at 8 AM, suggesting a demand for bikes during the morning rush hour.This suggests that many people use bikes to commute from home to work.

In [ ]:
df_temp = df.copy()  
df_temp['Holiday_Transformed'] = df_temp['Holiday'].map({"No Holiday": 0, "Holiday": 1})

# Visualisation
plt.figure(figsize=(10, 5))
plt.scatter(df_temp[df_temp['Holiday_Transformed'] == 0]['Hour'], df_temp[df_temp['Holiday_Transformed'] == 0]['Rented Bike Count'], label='Normal Day', alpha=0.7)
plt.scatter(df_temp[df_temp['Holiday_Transformed'] == 1]['Hour'], df_temp[df_temp['Holiday_Transformed'] == 1]['Rented Bike Count'], label='Holiday', alpha=0.7)
plt.xlabel('Hour')
plt.ylabel('Rented Bike Count')
plt.title('Hourly Distribution of Rented Bike Count: Vacation vs Non-Vacation Days')
plt.xticks(range(0, 24))
plt.legend()
plt.show()

The visualization reveals that bike rental counts are generally lower during vacation periods compared to non-vacation days. This suggests that the demand for rented bikes is influenced by holidays. The lower values during vacation periods may be attributed to a variety of factors, such as people not going to work, being away on trips, etc...

In [ ]:
plt.figure(figsize=(12, 5))
sns.scatterplot(x=df['Temperature(°C)'], y=df['Rented Bike Count'], alpha=0.5)
plt.title('Scatter Plot: Temperature vs Rented Bike Count')
plt.xlabel('Temperature')
plt.ylabel('Rented Bike Count')
plt.show()

In [ ]:
avg_rented_bikes_per_temp = df.groupby('Temperature(°C)')['Rented Bike Count'].mean().reset_index()

plt.figure(figsize=(10, 6))
plt.plot(avg_rented_bikes_per_temp['Temperature(°C)'], avg_rented_bikes_per_temp['Rented Bike Count'], marker='o')
plt.title('Average Rented Bike Count per Temperature')
plt.xlabel('Temperature')
plt.ylabel('Average Rented Bike Count')
plt.show()

In the last two visualizations wen can see that the demand for bikes decreases as the temperature drops. It reaches its highest point at around 30°C.

For extremely high temperatures, a decrease in the number of rental bicycles is observed.

In [ ]:
fig = px.scatter(df, x='Temperature(°C)', y='Rented Bike Count', color='Seasons', hover_data=['Humidity(%)', 'Wind speed (m/s)'], size='Rented Bike Count')
fig.update_layout(title='Overview of features')
fig.show()

This visualizations groups all we saw just before.

In [ ]:
plt.figure(figsize=(12, 5))
plt.scatter(x=df['Humidity(%)'], y=df['Rented Bike Count'], alpha=0.5)
plt.title('Humidity vs Rented Bike Count')
plt.xlabel('Humidity')
plt.ylabel('Rented Bike Count')
plt.show()

For extreme humidity values ​​the demand for bikes seems lower

In [ ]:
plt.figure(figsize=(12, 6))
plt.scatter(df['Visibility (10m)'], df['Rented Bike Count'], alpha=0.5)
plt.title('Visibility vs Rented Bike Count')
plt.xlabel('Visibility (10m)')
plt.ylabel('Rented Bike Count')
plt.show()

Quite naturally, the lower the visibility, the lower the demand for bikes.

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12, 5))

axes[0].scatter(df['Rainfall(mm)'], df['Rented Bike Count'])
axes[0].set(xlabel='Rainfall (mm)', ylabel='Rented Bike Count', title='Rainfall vs Rented Bike Count')

axes[1].scatter(df['Snowfall (cm)'], df['Rented Bike Count'])
axes[1].set(xlabel='Snowfall (cm)', ylabel='Rented Bike Count', title='Snowfall vs Rented Bike Count')
plt.show()


Both scatter plots exhibit a similar trend, showing that lower levels of rainfall and snowfall (approaching 0 cm) correspond to higher bike rental counts. This confirms that adverse weather conditions, such as heavy rain or snow, can act as deterrents for individuals to rent bikes. When faced with inclement weather, people may opt for alternative modes of transportation activities instead.

### Regulating skewness

In [ ]:
# Function to compare the skewness of the original data with the skewness of the transformed data
# Goal: to find the transformation that makes the data the most normal aka the least skewed

def compare_skew(df, column):
    fig, axes = plt.subplots(1, 4, figsize=(20,5))
    sns.distplot((df[column]), ax=axes[0]).set_title("Base data")
    sns.distplot(np.log1p(df[column]), ax=axes[1]).set_title("log1p")
    sns.distplot(np.sqrt(df[column]), ax=axes[2]).set_title("Square root")
    sns.distplot(np.cbrt(df[column]), ax=axes[3]).set_title("cube root")

In [ ]:
# On Rental Bike Count, we have a high skewness and kurtosis, so we will apply all 3 common transformations to see which one is the best
compare_skew(df_encoded, 'Rented Bike Count')

# There is a debate between the cube root and the square root transformation, but we will go with the square root
df_encoded['Rented Bike Count'] = np.sqrt(df_encoded['Rented Bike Count'])

In [ ]:
compare_skew(df_encoded, 'Wind speed (m/s)')

# The square root transformation is the best here
df_encoded['Wind speed (m/s)'] = np.sqrt(df_encoded['Wind speed (m/s)'])

In [ ]:
compare_skew(df_encoded, 'Visibility (10m)')

# No transformation can help here, which was to be expected
# We can expect the same thing from the rest of the columns with high skewness

### Defining input and output

In [ ]:
X = df_encoded.drop(['Rented Bike Count'], axis=1)
y = df_encoded['Rented Bike Count']

In [ ]:
# Defining train and test sets

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f'X_train = {X_train.shape}, X_test = {X_test.shape}')

# Modeling

In [ ]:
# We are trying to predict a continuous variable, so we will use regression models

In [ ]:
# Function to evaluate the performance of the models

results = pd.DataFrame(columns=['Model', 'R2', 'RMSE', 'MAE'])

def train_and_evaluate(model, name, scaler=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    if scaler:
        X_train = scaler.fit_transform(X_train)
        X_test = scaler.transform(X_test)
    model.fit(X_train, y_train)
    y_pred = model.predict(X_test)
    r2 = r2_score(y_test, y_pred)
    rmse = np.sqrt(mean_squared_error(y_test, y_pred))
    mae = mean_absolute_error(y_test, y_pred)
    print(name)
    print(f'R2: {r2}')
    print(f'RMSE: {rmse}')
    print(f'MAE: {mae}')

    global results
    if name not in results['Model'].values:
        results = pd.concat([results, pd.DataFrame([[name, r2, rmse, mae]], columns=['Model', 'R2', 'RMSE', 'MAE'])])

    plt.scatter(y_pred,y_test)
    plt.xlim(0, 40)
    plt.ylim(0, 40)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')

In [ ]:
train_and_evaluate(LinearRegression(), 'Linear Regression')

In [ ]:
train_and_evaluate(DecisionTreeRegressor(), 'Decision Tree')

In [ ]:
rf_params = {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}
train_and_evaluate(RandomForestRegressor(random_state=42, **rf_params), 'Random Forest')

In [ ]:
rf_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

rf_regressor = RandomForestRegressor(random_state=42)

#GridSearchCV
rf_grid_search = GridSearchCV(rf_regressor, rf_param_grid, scoring='r2', cv=3)
rf_grid_search.fit(X_train, y_train)

#best parameters
print("Best Hyperparameters for Random Forest:", rf_grid_search.best_params_)

#Retrain the model with the best parameters
best_rf_regressor = RandomForestRegressor(random_state=42, **rf_grid_search.best_params_)
train_and_evaluate(best_rf_regressor, 'Random Forest with Best Parameters')


In [ ]:
train_and_evaluate(LGBMRegressor(random_state=42), 'LGBM')

In [ ]:
param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [5, 10, 15],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0]
}

lgbm = LGBMRegressor(random_state=42)

grid_search = GridSearchCV(lgbm, param_grid, scoring='r2', cv=3)
grid_search.fit(X_train, y_train)

#best parameters
print("Best Hyperparameters:", grid_search.best_params_)

#Retrain the model with the best parameters
best_lgbm = LGBMRegressor(random_state=42, **grid_search.best_params_)
train_and_evaluate(best_lgbm, 'LGBM with Best Parameters')

In [ ]:
lgbm_params = {'colsample_bytree': 1.0, 'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}

train_and_evaluate(LGBMRegressor(random_state=42, **lgbm_params), 'LGBM SS', scaler=StandardScaler())

In [ ]:
train_and_evaluate(LGBMRegressor(random_state=42, **lgbm_params), 'LGBM MMS', scaler=MinMaxScaler())

In [ ]:
train_and_evaluate(LGBMRegressor(random_state=42, **lgbm_params), 'LGBM RS', scaler=RobustScaler())

In [ ]:
train_and_evaluate(SVR(), 'SVR SS', scaler=StandardScaler())

In [ ]:
train_and_evaluate(Lasso(), 'Lasso SS', scaler=StandardScaler())

In [ ]:
train_and_evaluate(Ridge(), 'Ridge SS', scaler=StandardScaler())

In [ ]:
grad_params = {'learning_rate': 0.1, 'max_depth': 7, 'n_estimators': 200, 'subsample': 1.0}

train_and_evaluate(GradientBoostingRegressor(random_state=42, **grad_params), 'Gradient Boosting')

In [ ]:
gb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7],
    'subsample': [0.8, 1.0]
}

gb_regressor = GradientBoostingRegressor(random_state=42)

gb_grid_search = GridSearchCV(gb_regressor, gb_param_grid, scoring='r2', cv=3)
gb_grid_search.fit(X_train, y_train)

print("Best Hyperparameters for Gradient Boosting with R2:", gb_grid_search.best_params_)

best_gb_regressor_r2 = GradientBoostingRegressor(random_state=42, **gb_grid_search.best_params_)
train_and_evaluate(best_gb_regressor_r2, 'Gradient Boosting with Best Parameters (R2)')

In [ ]:
train_and_evaluate(KNeighborsRegressor(), 'KNN')

In [ ]:
xt_params = {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}

train_and_evaluate(ExtraTreesRegressor(random_state=42, **xt_params), 'Extra Trees')

In [ ]:
et_param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

et_regressor = ExtraTreesRegressor(random_state=42)

et_grid_search = GridSearchCV(et_regressor, et_param_grid, scoring='r2', cv=3)
et_grid_search.fit(X_train, y_train)

print("Best Hyperparameters for Extra Trees:", et_grid_search.best_params_)

best_et_regressor = ExtraTreesRegressor(random_state=42, **et_grid_search.best_params_)
train_and_evaluate(best_et_regressor, 'Extra Trees with Best Parameters')

In [ ]:
# XGBRegressor

from xgboost import XGBRegressor

train_and_evaluate(XGBRegressor(random_state=42), 'XGBoost')

In [ ]:
xgb_param_grid = {
    'n_estimators': [50, 100, 200],
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [5, 10, 15],
    'subsample': [0.8, 1.0],
}

xgb_regressor = XGBRegressor(random_state=42)

xgb_grid_search = GridSearchCV(xgb_regressor, xgb_param_grid, scoring='r2', cv=3, verbose=2)
xgb_grid_search.fit(X_train, y_train)

print("Best Hyperparameters for XGBoost with R2:", xgb_grid_search.best_params_)

best_xgb_regressor_r2 = XGBRegressor(random_state=42, **xgb_grid_search.best_params_)
train_and_evaluate(best_xgb_regressor_r2, 'XGBoost with Best Parameters (R2)')

In [ ]:
xgb_params = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}
train_and_evaluate(XGBRegressor(random_state=42, **xgb_params), 'XGBoost GSCV RS', scaler=RobustScaler())

In [ ]:
xgb_params = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}
train_and_evaluate(XGBRegressor(random_state=42, **xgb_params), 'XGBoost GSCV SS', scaler=StandardScaler())

In [ ]:
xgb_params = {'learning_rate': 0.1, 'max_depth': 10, 'n_estimators': 200, 'subsample': 0.8}
train_and_evaluate(XGBRegressor(random_state=42, **xgb_params), 'XGBoost GSCV MMS', scaler=MinMaxScaler())

In [ ]:
display(results.sort_values(by='R2', ascending=False))